In [6]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

## XML File process
* Load all xml files and store them in a list
* Read xml files

In [32]:
xml_files = glob('datasets/Annotation/*/*.xml')

In [43]:
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    
    # extract filename
    image_name = f"{root.find('filename').text}.jpg"
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name, xmin, xmax, ymin, ymax])
        
    return parser

In [44]:
parser_all = list(map(extract_text, xml_files))

In [47]:
data = reduce(lambda x, y : x+y, parser_all)

In [49]:
df = pd.DataFrame(data, columns= ['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

In [50]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,n02097658_595.jpg,500,375,silky_terrier,82,378,7,355
1,n02097658_4869.jpg,375,500,silky_terrier,43,321,270,498
2,n02097658_26.jpg,448,400,silky_terrier,41,296,30,398
3,n02097658_422.jpg,500,333,silky_terrier,146,356,10,332
4,n02097658_9222.jpg,333,500,silky_terrier,0,331,12,498


In [51]:
df.shape

(22126, 8)

In [52]:
df['name'].value_counts()

name
Afghan_hound          287
Maltese_dog           264
Irish_wolfhound       263
Leonberg              256
Scottish_deerhound    246
                     ... 
Sussex_spaniel        153
malinois              153
Pekinese              152
groenendael           152
redbone               151
Name: count, Length: 120, dtype: int64